In [2]:
import pandas as pd, numpy as np

In [3]:
import os
os.chdir('E:\Assignments_SCMA632\Data')

In [4]:
df_ipl = pd.read_csv("IPL_ball_by_ball_updated till 2024.csv",low_memory=False)
salary = pd.read_excel("IPL SALARIES 2024.xlsx")

In [5]:
df_ipl.columns

Index(['Match id', 'Date', 'Season', 'Batting team', 'Bowling team',
       'Innings No', 'Ball No', 'Bowler', 'Striker', 'Non Striker',
       'runs_scored', 'extras', 'type of extras', 'score', 'score/wicket',
       'wicket_confirmation', 'wicket_type', 'fielders_involved',
       'Player Out'],
      dtype='object')

In [6]:
grouped_data = df_ipl.groupby(['Season', 'Innings No', 'Striker','Bowler']).agg({'runs_scored': sum, 'wicket_confirmation':sum}).reset_index()

In [7]:
grouped_data

,Season,Innings No,Striker,Bowler,runs_scored,wicket_confirmation
0,2007/08,1,A Chopra,DP Vijaykumar,1,0
1,2007/08,1,A Chopra,DW Steyn,1,1
2,2007/08,1,A Chopra,GD McGrath,2,0
3,2007/08,1,A Chopra,PJ Sangwan,6,1
4,2007/08,1,A Chopra,RP Singh,9,0
...,...,...,...,...,...,...
48781,2024,2,YBK Jaiswal,RJW Topley,0,1
48782,2024,2,YBK Jaiswal,SM Curran,6,0
48783,2024,2,YBK Jaiswal,Tilak Varma,5,0
48784,2024,2,YBK Jaiswal,VG Arora,10,1


In [8]:
total_runs_each_year = grouped_data.groupby(['Season', 'Striker'])['runs_scored'].sum().reset_index()
total_wicket_each_year = grouped_data.groupby(['Season', 'Bowler'])['wicket_confirmation'].sum().reset_index()

In [9]:
total_runs_each_year

,Season,Striker,runs_scored
0,2007/08,A Chopra,42
1,2007/08,A Kumble,13
2,2007/08,A Mishra,37
3,2007/08,A Mukund,0
4,2007/08,A Nehra,3
...,...,...,...
2593,2024,Vijaykumar Vyshak,1
2594,2024,WG Jacks,176
2595,2024,WP Saha,135
2596,2024,Washington Sundar,0


In [9]:
#pip install python-Levenshtein

In [10]:
from fuzzywuzzy import process

# Convert to DataFrame
df_salary = salary.copy()
df_runs = total_runs_each_year.copy()

# Function to match names
def match_names(name, names_list):
    match, score = process.extractOne(name, names_list)
    return match if score >= 80 else None  # Use a threshold score of 80

# Create a new column in df_salary with matched names from df_runs
df_salary['Matched_Player'] = df_salary['Player'].apply(lambda x: match_names(x, df_runs['Striker'].tolist()))

# Merge the DataFrames on the matched names
df_merged = pd.merge(df_salary, df_runs, left_on='Matched_Player', right_on='Striker')

In [11]:
df_original = df_merged.copy()

In [12]:
#susbsets data for last three years
df_merged = df_merged.loc[df_merged['Season'].isin(['2021', '2022', '2023'])]

In [13]:
df_merged.Season.unique()

array(['2023', '2022', '2021'], dtype=object)

In [14]:
df_merged.head()

,Player,Salary,Rs,international,iconic,Matched_Player,Season,Striker,runs_scored
0,Abhishek Porel,20 lakh,20,0,NaN,Abishek Porel,2023,Abishek Porel,33
3,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2022,A Nortje,1
4,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2023,A Nortje,37
13,Axar Patel,9 crore,900,0,NaN,AR Patel,2021,AR Patel,40
14,Axar Patel,9 crore,900,0,NaN,AR Patel,2022,AR Patel,182


In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [17]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_percentage_error
X = df_merged[['runs_scored']] # Independent variable(s)
y = df_merged['Rs'] # Dependent variable
# Split the data into training and test sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Create a LinearRegression model
model = LinearRegression()
# Fit the model on the training data
model.fit(X_train, y_train)

LinearRegression()

In [18]:
X.head()

,runs_scored
0,33
3,1
4,37
13,40
14,182


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Assuming df_merged is already defined and contains the necessary columns
X = df_merged[['runs_scored']] # Independent variable(s)
y = df_merged['Rs'] # Dependent variable

# Split the data into training and test sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add a constant to the model (intercept)
X_train_sm = sm.add_constant(X_train)

# Create a statsmodels OLS regression model
model = sm.OLS(y_train, X_train_sm).fit()

# Get the summary of the model
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                     Rs   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     15.83
Date:                Sun, 23 Jun 2024   Prob (F-statistic):           0.000100
Time:                        11:31:11   Log-Likelihood:                -1379.8
No. Observations:                 183   AIC:                             2764.
Df Residuals:                     181   BIC:                             2770.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         430.8473     46.111      9.344      

In [32]:
from fuzzywuzzy import process

# Convert to DataFrame
df_salary = salary.copy()
df_runs = total_wicket_each_year.copy()

# Function to match names
def match_names(name, names_list):
    match, score = process.extractOne(name, names_list)
    return match if score >= 80 else None  # Use a threshold score of 80

# Create a new column in df_salary with matched names from df_runs
df_salary['Matched_Player'] = df_salary['Player'].apply(lambda x: match_names(x, df_runs['Bowler'].tolist()))

# Merge the DataFrames on the matched names
df_merged = pd.merge(df_salary, df_runs, left_on='Matched_Player', right_on='Bowler')

In [37]:
df_merged[df_merged['wicket_confirmation']>10]

,Player,Salary,Rs,international,iconic,Matched_Player,Season,Bowler,wicket_confirmation
1,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2020/21,A Nortje,23
2,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2021,A Nortje,12
4,Anrich Nortje,6.5 crore,650,1,NaN,A Nortje,2023,A Nortje,11
6,Axar Patel,9 crore,900,0,NaN,AR Patel,2014,AR Patel,19
7,Axar Patel,9 crore,900,0,NaN,AR Patel,2015,AR Patel,14
...,...,...,...,...,...,...,...,...,...
589,T. Natarajan,3.2 crore,320,0,NaN,T Natarajan,2020/21,T Natarajan,19
591,T. Natarajan,3.2 crore,320,0,NaN,T Natarajan,2022,T Natarajan,20
592,T. Natarajan,3.2 crore,320,0,NaN,T Natarajan,2023,T Natarajan,13
593,T. Natarajan,3.2 crore,320,0,NaN,T Natarajan,2024,T Natarajan,13


In [26]:
#susbsets data for last three years
df_merged = df_merged.loc[df_merged['Season'].isin(['2022'])]

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Assuming df_merged is already defined and contains the necessary columns
X = df_merged[['wicket_confirmation']] # Independent variable(s)
y = df_merged['Rs'] # Dependent variable

# Split the data into training and test sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add a constant to the model (intercept)
X_train_sm = sm.add_constant(X_train)

# Create a statsmodels OLS regression model
model = sm.OLS(y_train, X_train_sm).fit()

# Get the summary of the model
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                     Rs   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.6043
Date:                Sun, 23 Jun 2024   Prob (F-statistic):              0.437
Time:                        11:39:37   Log-Likelihood:                -3701.4
No. Observations:                 484   AIC:                             7407.
Df Residuals:                     482   BIC:                             7415.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 480.9475    